In [ ]:
import pandas as pd
import json
import requests
import re
import dateparser
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Configurar o proxy TOR (SOCKS5)
proxies = {
    'http': 'socks5h://localhost:9050',
    'https': 'socks5h://localhost:9050'
}

with open('grupos.json', 'r') as file:
    data = json.load(file)

df = pd.DataFrame.from_dict(data, orient='index').explode('links_online')

df.reset_index(inplace=True)
df.columns = ['grupo', 'link_grupo', 'links_online']

# Adicionando as novas colunas
df['response_status_code'] = None 
df['html'] = None
df['infos'] = None
df['datas'] = None
df['qtd_datas'] = None




In [ ]:
df

In [ ]:
def extract_dates(html, days):
    # Padrões de datas que queremos capturar
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',           # AAAA-MM-DD
        r'\d{2}/\d{2}/\d{4}',           # DD/MM/AAAA ou MM/DD/AAAA (dependendo do contexto)
        r'\d{1,2} de \w+ de \d{4}',     # DD de Mês de AAAA
        r'\d{4}\.\d{2}\.\d{2}',         # AAAA.MM.DD
        r'\w+ \d{1,2}, \d{4}',          # Mês DD, AAAA
        r'\d{2}-\d{2}-\d{4}',           # DD-MM-AAAA
        r'\w{3} \d{1,2}, \d{4}',        # Mês Abreviado DD, AAAA
        r'\d{1,2} \w{3} \d{4}',         # DD Mon AAAA
        r'\d{4}/\d{2}/\d{2}',           # AAAA/MM/DD
        r'\d{8}',                       # AAAAMMDD ou DDMMYYYY (sem delimitadores)
        r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(?:\.\d+)?(?:Z|[+-]\d{2}:\d{2})',  # ISO 8601
        r'\w+ de \d{4}',                # Mês por Extenso e Ano
        r'\w+ \d{1,2}(?:st|nd|rd|th), \d{4}', # Mês e Dia por Extenso (Oct 1st, 2024)
        r'\d{4}\w{3}\d{2}',             # AAAA-MêsAbreviado-DD (2024Oct31)
        r'\d{2}/\d{2}/\d{2}',           # Dia/Mês/Ano Abreviado (10/12/24)
        r'\w{3}, \d{4}',                # Mês Abreviado e Ano (Oct, 2024)
        r'\d{4}-\d{2}',                 # Ano e Mês (2024-10)
        r'\d{4}, \w{3} \d{1,2}(?:st|nd|rd|th)', # Ano, Mês Abreviado e Dia por Extenso (2024, Oct 10th)
        
        # Formatos Internacionais
        r'\d{2}\.\d{2}\.\d{4}',                # Formato Alemão (TT.MM.JJJJ)
        r'\d{4}年\d{1,2}月\d{1,2}日',            # Formato Chinês (AAAA年MM月DD日)
]

    
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text()
    dates = []
    for pattern in date_patterns:
        found_dates = re.findall(pattern, text)
        dates.extend(found_dates)

    # Filtrar datas para mostrar apenas os últimos 7 dias
    collected_dates = []
    for date_str in dates:
        # Tentar converter a data de acordo com o formato
        for fmt in ['%Y-%m-%d', '%d/%m/%Y', '%d de %B de %Y', '%Y.%m.%d', '%B %d, %Y']:
            try:
                date = datetime.strptime(date_str, fmt)
                if (datetime.now() - timedelta(days)) <= date <= datetime.now():
                    collected_dates.append(date_str)
                break  # Se a data foi convertida com sucesso, saia do loop
            except ValueError:
                continue  # Tente o próximo formato

    return collected_dates

In [ ]:
days = 7

for index, row in df.iterrows():
    url = row['links_online']
    try:
        # Fazendo a requisição para o link usando requests
        response = requests.get(url, proxies=proxies, timeout=30)
        
        # Salvando o status code na coluna 'disponivel'
        df.at[index, 'response_status_code'] = response.status_code
        
        # Se o status code for 200, salvamos o HTML na coluna 'html'
        if response.status_code == 200:
            print("Acesso bem-sucedido!")
            df.at[index, 'html'] = response.text
            datas = extract_dates(response.text, days)
            if datas:
                df.at[index, 'infos'] = True
            else:
                df.at[index, 'infos'] = False
            df.at[index, 'datas'] = datas
            df.at[index, 'qtd_datas'] = len(datas)

    except requests.RequestException as e:
        # Em caso de erro na requisição, salvamos o status como None ou erro apropriado
        df.at[index, 'response_status_code'] = 'erro'




In [ ]:
df

In [ ]:
df.to_csv("raw_data.csv")


In [ ]:
# df = pd.read_csv("raw_data.csv", index_col=False)
df_disponivel = df[df['infos'] == True]
df_to_analize = df_disponivel.drop_duplicates(subset='html')
df_to_analize = df_disponivel.drop_duplicates(subset='grupo')
df_to_analize.reset_index(drop=True, inplace=True)

In [ ]:
df_to_analize.to_csv('df_to_analize.csv')

In [2]:
df_to_analize = pd.read_csv("df_to_analize.csv", index_col=False)

In [3]:
from grupos import Grupos

grupos = Grupos(days=7)

def processar_grupos(grupos, df_to_analize):
    resultados = []  # Lista para armazenar os resultados

    for index, row in df_to_analize.iterrows():
        try:
            grupo = row['grupo']
            html_content = row['html']
            
            # Chamar o método correspondente com base no valor da coluna 'grupo'
            if grupo == 'ransomhouse':
                result = grupos.ransomhouse(html_content)
            elif grupo == 'monti':
                result = grupos.monti(html_content)
            elif grupo == 'play':
                result = grupos.play(html_content)
            elif grupo == 'handala':
                result = grupos.handala(html_content)
            elif grupo == 'blackbyte':
                result = grupos.blackbyte(html_content)
            else:
                print(f"Grupo '{grupo}' ainda não possui um scraper específico. Link para obter o html: {row['links_online']}")
                continue  # Pular para o próximo grupo se não houver scraper

            # Adiciona os resultados à lista, incluindo o grupo como uma nova chave
            for item in result:
                item['grupo'] = grupo  # Adicionando o grupo ao resultado
                resultados.append(item)  # Adicionando cada resultado à lista

        except Exception as e:
            print(f"Erro ao processar o grupo '{grupo}': {e}")

    return resultados

In [5]:

data = processar_grupos(grupos, df_to_analize)

In [9]:
df = pd.DataFrame(data)

# Agrupar por grupo e contar as ocorrências
ranking = df.groupby('grupo').size().reset_index(name='count')

# Ordenar pelo número de ocorrências em ordem decrescente
ranking = ranking.sort_values(by='count', ascending=False)

# Criar uma coluna de ranking
ranking['rank'] = ranking['count'].rank(method='min', ascending=False).astype(int)


       grupo  count  rank
3       play     12     1
0  blackbyte      7     2
1    handala      4     3
2      monti      1     4


In [8]:
ranking.reset_index()

,index,grupo,count,rank
0,3,play,12,1
1,0,blackbyte,7,2
2,1,handala,4,3
3,2,monti,1,4
